# Projeto Final CODENATION::Data Science
### Autor: José Ângelo de Assis Júnior

## Objetivo Principal
Criar um sistema automatizado que recomenda leads a partir de uma lista

## Objetivos deste Notebook
Parte 2 da criação de modelo de Machine Learning capaz de classificar cliente

In [ ]:
import numpy  as np
import pandas as pd

# recarregar os dados
from google.colab import drive
drive.mount('/content/drive/')

mercado    = pd.read_csv('/content/drive/My Drive/codenation/codeNation/Projeto/data/mercado.csv')
portfolio1 = pd.read_csv('/content/drive/My Drive/codenation/codeNation/Projeto/data/portfolio1.csv')
portfolio2 = pd.read_csv('/content/drive/My Drive/codenation/codeNation/Projeto/data/portfolio2.csv')
portfolio3 = pd.read_csv('/content/drive/My Drive/codenation/codeNation/Projeto/data/portfolio3.csv')

mercado.shape, portfolio1.shape, portfolio2.shape, portfolio3.shape

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


((462298, 43), (555, 43), (566, 43), (265, 43))

Treinamento do modelo com o melhor número de clusters identificado na parte 1

In [ ]:
from sklearn.cluster import KMeans

X = mercado.drop(['sg_uf', 'id'], axis=1)

kmeans = KMeans(n_clusters = 4, init = 'random', n_init = 50, algorithm = 'full').fit(X)

Gerar os clusters para os conjuntos de dados

In [ ]:
mercado_clusters    = kmeans.predict(X)
portfolio1_clusters = kmeans.predict(portfolio1.drop(['sg_uf', 'id'], axis=1))
portfolio2_clusters = kmeans.predict(portfolio2.drop(['sg_uf', 'id'], axis=1))
portfolio3_clusters = kmeans.predict(portfolio3.drop(['sg_uf', 'id'], axis=1))

Calcular as distâncias até os clusters

In [ ]:
mercado_dist    = kmeans.transform(X)
portfolio1_dist = kmeans.transform(portfolio1.drop(['sg_uf', 'id'], axis=1))
portfolio2_dist = kmeans.transform(portfolio2.drop(['sg_uf', 'id'], axis=1))
portfolio3_dist = kmeans.transform(portfolio3.drop(['sg_uf', 'id'], axis=1))

In [ ]:
mercado['cluster'] = mercado_clusters

In [ ]:
mercado['dist_centroide'] = np.amin(mercado_dist, 1)

In [ ]:
mercado.head()

,sg_uf,id,idade_empresa_anos,fl_email,fl_telefone,fl_rm,fl_veiculo,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,qt_coligados,qt_ufs_coligados,qt_ramos_coligados,media_vl_folha_coligados,media_vl_folha_coligados_gp,media_faturamento_est_coligados,qt_coligadas,sum_faturamento_estimado_coligadas,vl_faturamento_estimado_grupo_aux,qt_filiais,qt_total_socios_pj,de_saude_tributaria_VERDE,de_saude_tributaria_CINZA,de_saude_tributaria_AMARELO,de_saude_tributaria_LARANJA,de_saude_tributaria_AZUL,de_saude_tributaria_VERMELHO,de_nivel_atividade_ALTA,de_nivel_atividade_BAIXA,de_nivel_atividade_MEDIA,de_saude_rescencia_ACIMA DE 1 ANO,de_saude_rescencia_ATE 1 ANO,de_saude_rescencia_SEM INFORMACAO,de_saude_rescencia_ATE 6 MESES,Area_Territorial,População_estimada,Densidade_demográfica,Matrículas_ensino_fundamental,IDH_2010,Receitas_realizadas_2017,Despesas_empenhadas_2017,Rendimento_mensal_domiciliar_per_capita_2019,Total_veIculos_2018,cluster,dist_centroide
0,AC,5124a4ed965efeccdf43cd402f05d127f5e87fcc2d88f7...,0.006823,True,True,False,False,0.000000,0.000000,0.348485,0.001183,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,7.584469e-07,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.073896,0.0,0.038781,0.0,0.470588,0.0,0.0,0.508,0.0,2,1.819693
1,AC,8a584390e7b2fba5ef33a51b2f8fd63bd202194db3e5ee...,0.064801,True,True,False,False,0.003987,0.003194,0.727273,0.027219,0.2,0.186047,0.054234,0.003665,0.023703,0.0,0.0,1.000000e+00,0.796828,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.073896,0.0,0.038781,0.0,0.470588,0.0,0.0,0.508,0.0,3,2.061845
2,AC,140241eb6bdccbf0ba69739cdb9ca1e0bf4e795125aee2...,0.069384,False,True,False,False,0.000000,0.000000,0.348485,0.001183,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,7.584469e-07,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.073896,0.0,0.038781,0.0,0.470588,0.0,0.0,0.508,0.0,3,1.732713
3,AC,c5e70d47fe82cf519287e51c21d9557f821c74dde25eb2...,0.312085,False,False,False,False,0.028931,0.017479,0.818182,0.001183,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.619387e-01,0.057945,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.073896,0.0,0.038781,0.0,0.470588,0.0,0.0,0.508,0.0,1,1.787742
4,AC,2a034ebd67224b1b823422411890c5adb6e954475a232b...,0.031281,True,True,False,False,0.000000,0.000000,0.348485,0.001183,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,4.019079e-08,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.073896,0.0,0.038781,0.0,0.470588,0.0,0.0,0.508,0.0,3,1.565848


In [ ]:
mercado.id[5]

'6953e6aa0e4a447a84048a9c46ab897f60a136eb311490db1e06d684b9fc2456'

In [ ]:
# Captura o cluster do id
cluster_do_id     = mercado[mercado['id'] == '6953e6aa0e4a447a84048a9c46ab897f60a136eb311490db1e06d684b9fc2456']['cluster'].values[0]

# Captura a distância do id para o centroide de seu cluster
dist_centroide_id = mercado[mercado['id'] == '6953e6aa0e4a447a84048a9c46ab897f60a136eb311490db1e06d684b9fc2456']['dist_centroide'].values[0]

cluster_do_id, dist_centroide_id

(3, 1.7181289709106713)

In [ ]:
# Todo o conjunto de dados do cluster do id selecionado
distancias_cluster = mercado[mercado['cluster'] == cluster_do_id]['dist_centroide']

In [ ]:
# índices dos ids com distâncias mais próximas maiores do que do id selecionado
distancias_cluster[distancias_cluster > dist_centroide_id].sort_values().head()

308691    1.718133
375396    1.718134
54448     1.718142
356025    1.718156
28458     1.718158
Name: dist_centroide, dtype: float64

In [ ]:
# índices dos ids com distâncias mais próximas menores do que do id selecionado
distancias_cluster[distancias_cluster < dist_centroide_id].sort_values(ascending=False).head()

346128    1.718119
358899    1.718115
53436     1.718115
30753     1.718107
21403     1.718107
Name: dist_centroide, dtype: float64

In [ ]:
# selecionando íncides dos 5 mais próximos com distâncias menores:
indices_selecionados_menores = distancias_cluster[distancias_cluster < dist_centroide_id].sort_values(ascending=False).head().index

# selecionando íncides dos 5 mais próximos com distâncias maiores:
indices_selecionados_maiores = distancias_cluster[distancias_cluster > dist_centroide_id].sort_values().head().index

mercado.iloc[indices_selecionados_menores]

,sg_uf,id,idade_empresa_anos,fl_email,fl_telefone,fl_rm,fl_veiculo,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,qt_coligados,qt_ufs_coligados,qt_ramos_coligados,media_vl_folha_coligados,media_vl_folha_coligados_gp,media_faturamento_est_coligados,qt_coligadas,sum_faturamento_estimado_coligadas,vl_faturamento_estimado_grupo_aux,qt_filiais,qt_total_socios_pj,de_saude_tributaria_VERDE,de_saude_tributaria_CINZA,de_saude_tributaria_AMARELO,de_saude_tributaria_LARANJA,de_saude_tributaria_AZUL,de_saude_tributaria_VERMELHO,de_nivel_atividade_ALTA,de_nivel_atividade_BAIXA,de_nivel_atividade_MEDIA,de_saude_rescencia_ACIMA DE 1 ANO,de_saude_rescencia_ATE 1 ANO,de_saude_rescencia_SEM INFORMACAO,de_saude_rescencia_ATE 6 MESES,Area_Territorial,População_estimada,Densidade_demográfica,Matrículas_ensino_fundamental,IDH_2010,Receitas_realizadas_2017,Despesas_empenhadas_2017,Rendimento_mensal_domiciliar_per_capita_2019,Total_veIculos_2018,cluster,dist_centroide
346128,RN,1ee349240ac091ca8d9cc3ed314d1749396a4b616090e1...,0.167679,False,False,True,False,0.0,0.000000,0.727273,0.001183,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.584469e-07,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.423836,1.0,0.303517,0.882353,0.580830,0.454531,0.842,0.714008,3,1.718119
358899,RN,295014abb3bdbb464dba21e3908d7e1910e5cadd8e1b41...,0.112044,True,False,True,False,0.0,0.000000,0.348485,0.001183,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.665820e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.423836,1.0,0.303517,0.882353,0.580830,0.454531,0.842,0.714008,3,1.718115
53436,AM,49f01b76d5e6285c23a872eb1436225dd3edca8aa85753...,0.054323,False,False,True,True,0.0,0.000008,0.393939,0.011834,0.08,0.069767,0.010496,0.000855,0.000340,0.011827,0.003048,7.584469e-07,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.526810,0.0,0.535944,0.686275,0.901031,0.800544,0.412,0.426579,3,1.718115
30753,AM,4a7ae63df3bc4a5486ec61adb2870e5b194925fc21e519...,0.045132,True,False,True,True,0.0,0.000008,0.333333,0.016568,0.12,0.081395,0.001506,0.000080,0.000079,0.017083,0.001019,1.535644e-05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.526810,0.0,0.535944,0.686275,0.901031,0.800544,0.412,0.426579,3,1.718107
21403,AM,aee2f13dafc66718cd13245006106f3dda8fe0b9cd5ca7...,0.206606,False,False,True,True,0.0,0.000016,0.363636,0.001183,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.584469e-07,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.526810,0.0,0.535944,0.686275,0.901031,0.800544,0.412,0.426579,3,1.718107


In [ ]:
type(indices_selecionados_maiores)

pandas.core.indexes.numeric.Int64Index

Reimportar os dados brutos e fazer os pré-tratamentos antes da identificação do melhor número de clusters (notebook parte 1). Depois, gerar novamente os gráficos. Assim, poderemos analisar se a modelagem foi efetiva

In [ ]:
ls_cols = ['id', 'sg_uf', 'idade_empresa_anos', 'fl_st_especial', 'fl_email', 'fl_telefone', 'fl_rm', 'fl_spa', 'fl_antt', 
           'vl_total_tancagem', 'vl_total_veiculos_antt', 'qt_art', 'vl_total_veiculos_pesados_grupo', 'vl_total_veiculos_leves_grupo', 
           'vl_total_tancagem_grupo', 'vl_total_veiculos_antt_grupo', 'sg_uf_matriz', 'de_saude_tributaria', 'de_nivel_atividade', 
           'de_indicador_telefone', 'fl_simples_irregular', 'vl_frota', 'empsetorcensitariofaixarendapopulacao', 'qt_socios_st_regular', 
           'qt_socios_st_suspensa', 'qt_socios_pep', 'qt_socios_pj_nulos', 'qt_socios_pj_baixados', 'qt_socios_pj_suspensos', 'qt_socios_pj_inaptos', 
           'vl_idade_media_socios_pj', 'qt_coligados', 'qt_socios_coligados', 'qt_coligados_matriz', 'qt_coligados_baixada', 'qt_coligados_inapta', 
           'qt_coligados_suspensa', 'qt_coligados_nula', 'idade_media_coligadas_baixadas', 'qt_ufs_coligados', 'qt_regioes_coligados', 
           'qt_ramos_coligados', 'qt_funcionarios_coligados', 'media_funcionarios_coligados_gp', 'media_vl_folha_coligados_gp', 
           'media_faturamento_est_coligados', 'media_faturamento_est_coligados_gp', 'qt_coligados_atividade_alto', 'qt_coligados_atividade_medio', 
           'qt_coligados_atividade_baixo', 'qt_coligados_atividade_mt_baixo', 'qt_coligados_atividade_inativo', 'sum_faturamento_estimado_coligadas', 
           'vl_faturamento_estimado_aux', 'vl_faturamento_estimado_grupo_aux', 'qt_funcionarios_grupo', 'total', 'qt_admitidos_12meses', 
           'qt_desligados_12meses', 'qt_desligados', 'qt_admitidos', 'media_meses_servicos_all', 'media_meses_servicos', 'qt_funcionarios', 
           'qt_funcionarios_12meses', 'tx_crescimento_12meses', 'tx_crescimento_24meses', 'tx_rotatividade', 'qt_filiais', 'qt_socios_pj_ativos',
           'media_vl_folha_coligados', 'qt_coligadas', 'qt_socios_pf', 'qt_socios_pj', 'de_faixa_faturamento_estimado', 'vl_faturamento_estimado_aux',
           'fl_optante_simples', 'fl_optante_simei', 'qt_coligados_ativo', 'nu_meses_rescencia', 'de_saude_rescencia', 'fl_veiculo']

mercado    = pd.read_csv('/content/drive/My Drive/codenation/codeNation/Projeto/data/estaticos_market.csv', usecols=ls_cols)
portfolio1 = pd.read_csv('/content/drive/My Drive/codenation/codeNation/Projeto/data/estaticos_portfolio1.csv', usecols=ls_cols)
portfolio2 = pd.read_csv('/content/drive/My Drive/codenation/codeNation/Projeto/data/estaticos_portfolio2.csv')
portfolio3 = pd.read_csv('/content/drive/My Drive/codenation/codeNation/Projeto/data/estaticos_portfolio3.csv')

portfolio2.drop('Unnamed: 0', axis=1, inplace = True)
portfolio3.drop('Unnamed: 0', axis=1, inplace = True)

# complementar o portfolio2

# 1. ajustar o índice dos dataframes para executar o merge
mercado.set_index('id', inplace=True)
portfolio2.set_index('id', inplace=True)

# 2. executar o merge
portfolio2 = portfolio2.join(mercado)

# 3. reajustar o índice do conjunto de dados
portfolio2.reset_index(inplace=True)
mercado.reset_index(inplace=True)

# complementar o portfolio3

# 1. ajustar o índice dos dataframes para executar o merge
mercado.set_index('id', inplace=True)
portfolio3.set_index('id', inplace=True)

# 2. executar o merge
portfolio3 = portfolio3.join(mercado)

# 3. reajustar o índice do conjunto de dados
portfolio3.reset_index(inplace=True)
mercado.reset_index(inplace=True)

In [ ]:
# função para identificar dados ausentes
# fonte: https://www.kaggle.com/cgump3rt/investigate-missing-values

def missing_values_table(df, show=False):
    mis_val = df.isnull().sum()
    
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    
    print ("Your dataframe has " + str(df.shape[1]) + " columns. "
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns with missing values.\n")
    
    if show == True:
        null_counts = df.isnull().sum()/len(df)
        plt.figure(figsize=(28, 7))
        plt.xticks(np.arange(len(null_counts))+0.5,null_counts.index,rotation='vertical')
        plt.ylabel('fraction of rows with missing data')
        plt.axhline(y=0.1, color='r', linestyle='-')
        plt.axhline(y=0.05, color='g', linestyle='-')
        plt.bar(np.arange(len(null_counts)),null_counts)
    
    return mis_val_table_ren_columns

In [ ]:
# corrigir o valor SEM INFORMACAO na coluna de_faixa_faturamento_estimado
def col_de_faixa_faturamento_estimado(df):
    if df['de_faixa_faturamento_estimado'] == 'SEM INFORMACAO':
        if df['vl_faturamento_estimado_aux'] <= 81000:
            return 'ATE R$ 81.000,00'
        elif df['vl_faturamento_estimado_aux'] <= 36000:
            return 'DE R$ 81.000,01 A R$ 360.000,00'
        elif df['vl_faturamento_estimado_aux'] <= 1500000:
            return 'DE R$ 360.000,01 A R$ 1.500.000,00'
        elif df['vl_faturamento_estimado_aux'] <= 4800000:
            return 'DE R$ 1.500.000,01 A R$ 4.800.000,00'
        elif df['vl_faturamento_estimado_aux'] <= 30000000:
            return 'DE R$ 10.000.000,01 A R$ 30.000.000,00'
        elif df['vl_faturamento_estimado_aux'] <= 100000000:
            return 'DE R$ 30.000.000,01 A R$ 100.000.000,00'
        elif df['vl_faturamento_estimado_aux'] <= 300000000:
            return 'DE R$ 100.000.000,01 A R$ 300.000.000,00'
        elif df['vl_faturamento_estimado_aux'] <= 500000000:
            return 'DE R$ 300.000.000,01 A R$ 500.000.000,00'
        elif df['vl_faturamento_estimado_aux'] <= 1000000000:
            return 'DE R$ 500.000.000,01 A 1 BILHAO DE REAIS'
        elif df['vl_faturamento_estimado_aux'] > 1000000000:
            return 'ACIMA DE 1 BILHAO DE REAIS'
    else:
        x = df['de_faixa_faturamento_estimado']
        return x

# aplicando a função
mercado['de_faixa_faturamento_estimado']    = mercado.apply(col_de_faixa_faturamento_estimado, axis=1)

if portfolio1['de_faixa_faturamento_estimado'].isna().sum() > 0:
    portfolio1['de_faixa_faturamento_estimado'] = portfolio1.apply(col_de_faixa_faturamento_estimado, axis=1)

if portfolio2['de_faixa_faturamento_estimado'].isna().sum() > 0:
    portfolio2['de_faixa_faturamento_estimado'] = portfolio2.apply(col_de_faixa_faturamento_estimado, axis=1)

if portfolio3['de_faixa_faturamento_estimado'].isna().sum() > 0:
    portfolio3['de_faixa_faturamento_estimado'] = portfolio3.apply(col_de_faixa_faturamento_estimado, axis=1)

In [ ]:
# reduzir ou eliminiar os valores ausentes e incorretos na coluna fl_optante_simei 
# Obs.: para ser tributado pelo SIMEI, o faturamento é de até 81.000,00
def col_fl_optante_simei(df):
    if np.isnan(df['fl_optante_simei']):
        if df['de_faixa_faturamento_estimado'] == 'ATE R$ 81.000,00':
            return True
        elif df['vl_faturamento_estimado_aux'] <= 81000:
            return True
        else:
            return False
    else:
        if df['fl_optante_simei'] == True:
            return True
        else:
            return False

# aplicando a função
mercado['fl_optante_simei'] = mercado.apply(col_fl_optante_simei, axis=1)

if portfolio1['fl_optante_simei'].isna().sum() > 0:
    portfolio1['fl_optante_simei'] = portfolio1.apply(col_fl_optante_simei, axis=1)

if portfolio2['fl_optante_simei'].isna().sum() > 0:
    portfolio2['fl_optante_simei'] = portfolio2.apply(col_fl_optante_simei, axis=1)

if portfolio3['fl_optante_simei'].isna().sum() > 0:
    portfolio3['fl_optante_simei'] = portfolio3.apply(col_fl_optante_simei, axis=1)

In [ ]:
# reduzir ou eliminiar os valores ausentes na coluna qt_socios_st_regular
def col_qt_socios_st_regular(df):
    if np.isnan(df['qt_socios_st_regular']):
        x = df['qt_socios_pf'] + df['qt_socios_pj']
    else:
        x = df['qt_socios_st_regular']

    return x

# aplicando a função
mercado['qt_socios_st_regular'] = mercado.apply(col_qt_socios_st_regular, axis=1)

if portfolio1['qt_socios_st_regular'].isna().sum() > 0:
    portfolio1['qt_socios_st_regular'] = portfolio1.apply(col_qt_socios_st_regular, axis=1)

if portfolio2['qt_socios_st_regular'].isna().sum() > 0:
    portfolio2['qt_socios_st_regular'] = portfolio2.apply(col_qt_socios_st_regular, axis=1)

if portfolio3['qt_socios_st_regular'].isna().sum() > 0:
    portfolio3['qt_socios_st_regular'] = portfolio3.apply(col_qt_socios_st_regular, axis=1)

In [ ]:
df_missing = missing_values_table(mercado)

ls_cols_name = list(df_missing.index)
ls_cols_coligadas = []

for col in mercado.columns:
    if 'coligad' in col and col in ls_cols_name:
        ls_cols_coligadas.append(col)

mercado.drop('idade_media_coligadas_baixadas', axis=1, inplace=True)
portfolio1.drop('idade_media_coligadas_baixadas', axis=1, inplace=True)
portfolio2.drop('idade_media_coligadas_baixadas', axis=1, inplace=True)
portfolio3.drop('idade_media_coligadas_baixadas', axis=1, inplace=True)

In [ ]:
# substituir os valores nulos por zero na coluna qt_coligados, com a seguinte regra:
# considerando que o Simples Nacional tem como público empresas de até 4,8 milhões/ano
# e que o SIMEI tem como público empresa de até 81.000/ano, essas empresas enquadradas
# neste perfil terão os valores nulos substituídos por zero
def col_qt_coligadas(df):
    if pd.isnull(df['qt_coligados']):
        if df['de_faixa_faturamento_estimado'] == 'ATE R$ 81.000,00' or \
        df['de_faixa_faturamento_estimado'] == 'DE R$ 81.000,01 A R$ 360.000,00' or \
        df['de_faixa_faturamento_estimado'] == 'DE R$ 360.000,01 A R$ 1.500.000,00' or \
        df['de_faixa_faturamento_estimado'] == 'DE R$ 1.500.000,01 A R$ 4.800.000,00':
            return 0
        elif df['vl_faturamento_estimado_aux'] <= 4800000:
            return 0
        elif df['fl_optante_simples'] == True:
            return 0
        elif df['fl_optante_simei'] == True:
            return 0
        else:
            return np.nan
    else:
        x = df['qt_coligados']
        return x

# aplicando a função
mercado['qt_coligados'] = mercado.apply(col_qt_coligadas, axis=1)

if portfolio1['qt_coligados'].isna().sum() > 0:
    portfolio1['qt_coligados'] = portfolio1.apply(col_qt_coligadas, axis=1)

if portfolio2['qt_coligados'].isna().sum() > 0:
    portfolio2['qt_coligados'] = portfolio2.apply(col_qt_coligadas, axis=1)

if portfolio3['qt_coligados'].isna().sum() > 0:
    portfolio3['qt_coligados'] = portfolio3.apply(col_qt_coligadas, axis=1)

In [ ]:
# substituir os valores ausentes por valor fixo
mercado['qt_coligados'].fillna(-1, inplace=True)
portfolio1['qt_coligados'].fillna(-1, inplace=True)
portfolio2['qt_coligados'].fillna(-1, inplace=True)
portfolio3['qt_coligados'].fillna(-1, inplace=True)

# atualizar a lista de variáveis referente a coligas
df_missing = missing_values_table(mercado)

ls_cols_name = list(df_missing.index)
ls_cols_coligadas = []

for col in mercado.columns:
    if 'coligad' in col and col in ls_cols_name:
        ls_cols_coligadas.append(col)

# como todas as variáveis possuem o mesmo dtype, declarar uma única função
# para tratar dados ausentes é suficiente
for col_coligada in ls_cols_coligadas:
    def col_qt_coligadas(df):
        if pd.isnull(df[col_coligada]):
            if df['qt_coligados'] == 0:
                return 0
        else:
            x = df[col_coligada]
            return x

    # aplicando a função
    mercado[col_coligada]    = mercado.apply(col_qt_coligadas, axis=1)

    if portfolio1[col_coligada].isna().sum() > 0:
        portfolio1[col_coligada] = portfolio1.apply(col_qt_coligadas, axis=1)
    
    if portfolio2[col_coligada].isna().sum() > 0:
        portfolio2[col_coligada] = portfolio2.apply(col_qt_coligadas, axis=1)
    
    if portfolio3[col_coligada].isna().sum() > 0:
        portfolio3[col_coligada] = portfolio3.apply(col_qt_coligadas, axis=1)

In [ ]:
# conjunto de dados mercado
mercado['qt_coligados_ativo'].fillna(0, inplace=True)
mercado['qt_coligados_baixada'].fillna(0, inplace=True)
mercado['qt_coligados_inapta'].fillna(0, inplace=True)
mercado['qt_coligados_suspensa'].fillna(0, inplace=True)
mercado['qt_coligados_nula'].fillna(0, inplace=True)
mercado['media_vl_folha_coligados'].fillna(0, inplace=True)
mercado['media_vl_folha_coligados_gp'].fillna(0, inplace=True)
mercado['media_faturamento_est_coligados'].fillna(0, inplace=True)
mercado['media_faturamento_est_coligados_gp'].fillna(0, inplace=True)
mercado['qt_coligados_atividade_baixo'].fillna(0, inplace=True)
mercado['qt_coligados_atividade_mt_baixo'].fillna(0, inplace=True)
mercado['qt_coligados_atividade_inativo'].fillna(0, inplace=True)
mercado['qt_coligadas'].fillna(0, inplace=True)
mercado['sum_faturamento_estimado_coligadas'].fillna(0, inplace=True)

# conjunto de dados portfolio1
portfolio1['qt_coligados_ativo'].fillna(0, inplace=True)
portfolio1['qt_coligados_baixada'].fillna(0, inplace=True)
portfolio1['qt_coligados_inapta'].fillna(0, inplace=True)
portfolio1['qt_coligados_suspensa'].fillna(0, inplace=True)
portfolio1['qt_coligados_nula'].fillna(0, inplace=True)
portfolio1['media_vl_folha_coligados'].fillna(0, inplace=True)
portfolio1['media_vl_folha_coligados_gp'].fillna(0, inplace=True)
portfolio1['media_faturamento_est_coligados'].fillna(0, inplace=True)
portfolio1['media_faturamento_est_coligados_gp'].fillna(0, inplace=True)
portfolio1['qt_coligados_atividade_baixo'].fillna(0, inplace=True)
portfolio1['qt_coligados_atividade_mt_baixo'].fillna(0, inplace=True)
portfolio1['qt_coligados_atividade_inativo'].fillna(0, inplace=True)
portfolio1['qt_coligadas'].fillna(0, inplace=True)
portfolio1['sum_faturamento_estimado_coligadas'].fillna(0, inplace=True)

# conjunto de dados portfolio2
portfolio2['qt_coligados_ativo'].fillna(0, inplace=True)
portfolio2['qt_coligados_baixada'].fillna(0, inplace=True)
portfolio2['qt_coligados_inapta'].fillna(0, inplace=True)
portfolio2['qt_coligados_suspensa'].fillna(0, inplace=True)
portfolio2['qt_coligados_nula'].fillna(0, inplace=True)
portfolio2['media_vl_folha_coligados'].fillna(0, inplace=True)
portfolio2['media_vl_folha_coligados_gp'].fillna(0, inplace=True)
portfolio2['media_faturamento_est_coligados'].fillna(0, inplace=True)
portfolio2['media_faturamento_est_coligados_gp'].fillna(0, inplace=True)
portfolio2['qt_coligados_atividade_baixo'].fillna(0, inplace=True)
portfolio2['qt_coligados_atividade_mt_baixo'].fillna(0, inplace=True)
portfolio2['qt_coligados_atividade_inativo'].fillna(0, inplace=True)
portfolio2['qt_coligadas'].fillna(0, inplace=True)
portfolio2['sum_faturamento_estimado_coligadas'].fillna(0, inplace=True)

# conjunto de dados portfolio3
portfolio3['qt_coligados_ativo'].fillna(0, inplace=True)
portfolio3['qt_coligados_baixada'].fillna(0, inplace=True)
portfolio3['qt_coligados_inapta'].fillna(0, inplace=True)
portfolio3['qt_coligados_suspensa'].fillna(0, inplace=True)
portfolio3['qt_coligados_nula'].fillna(0, inplace=True)
portfolio3['media_vl_folha_coligados'].fillna(0, inplace=True)
portfolio3['media_vl_folha_coligados_gp'].fillna(0, inplace=True)
portfolio3['media_faturamento_est_coligados'].fillna(0, inplace=True)
portfolio3['media_faturamento_est_coligados_gp'].fillna(0, inplace=True)
portfolio3['qt_coligados_atividade_baixo'].fillna(0, inplace=True)
portfolio3['qt_coligados_atividade_mt_baixo'].fillna(0, inplace=True)
portfolio3['qt_coligados_atividade_inativo'].fillna(0, inplace=True)
portfolio3['qt_coligadas'].fillna(0, inplace=True)
portfolio3['sum_faturamento_estimado_coligadas'].fillna(0, inplace=True)

In [ ]:
# excluir colunas referente a coligadas cuja proporção de valores ausentes ultrapasse 10%
ls_cols_coligadas = []

for col in mercado.columns:
    if 'coligad' in col and col in ls_cols_name:
        ls_cols_coligadas.append(col)

df_missing = missing_values_table(mercado[ls_cols_coligadas])
ls_cols_coligada_10 = df_missing[df_missing['% of Total Values'] > 10].index

mercado.drop(ls_cols_coligada_10, axis=1, inplace=True)
portfolio1.drop(ls_cols_coligada_10, axis=1, inplace=True)
portfolio2.drop(ls_cols_coligada_10, axis=1, inplace=True)
portfolio3.drop(ls_cols_coligada_10, axis=1, inplace=True)

In [ ]:
mercado['qt_socios_pj_nulos'].fillna(0, inplace=True)
mercado['qt_socios_pj_baixados'].fillna(0, inplace=True)
mercado['qt_socios_pj_ativos'].fillna(0, inplace=True)
mercado['qt_socios_pj_suspensos'].fillna(0, inplace=True)
mercado['qt_socios_pj_inaptos'].fillna(0, inplace=True)

portfolio1['qt_socios_pj_nulos'].fillna(0, inplace=True)
portfolio1['qt_socios_pj_baixados'].fillna(0, inplace=True)
portfolio1['qt_socios_pj_ativos'].fillna(0, inplace=True)
portfolio1['qt_socios_pj_suspensos'].fillna(0, inplace=True)
portfolio1['qt_socios_pj_inaptos'].fillna(0, inplace=True)

portfolio2['qt_socios_pj_nulos'].fillna(0, inplace=True)
portfolio2['qt_socios_pj_baixados'].fillna(0, inplace=True)
portfolio2['qt_socios_pj_ativos'].fillna(0, inplace=True)
portfolio2['qt_socios_pj_suspensos'].fillna(0, inplace=True)
portfolio2['qt_socios_pj_inaptos'].fillna(0, inplace=True)

portfolio3['qt_socios_pj_nulos'].fillna(0, inplace=True)
portfolio3['qt_socios_pj_baixados'].fillna(0, inplace=True)
portfolio3['qt_socios_pj_ativos'].fillna(0, inplace=True)
portfolio3['qt_socios_pj_suspensos'].fillna(0, inplace=True)
portfolio3['qt_socios_pj_inaptos'].fillna(0, inplace=True)

In [ ]:
# lista de colunas com mais de 90% de valores ausentes
df_missing = missing_values_table(mercado)
ls_cols_missin_90 = df_missing[df_missing['% of Total Values'] > 90].index

# remocao dessas colunas dos datasets
mercado.drop(ls_cols_missin_90, axis=1, inplace=True)
portfolio1.drop(ls_cols_missin_90, axis=1, inplace=True)
portfolio2.drop(ls_cols_missin_90, axis=1, inplace=True)
portfolio3.drop(ls_cols_missin_90, axis=1, inplace=True)

In [ ]:
# conjunto de dados mercado
mercado['qt_total_socios_pj']      = mercado.qt_socios_pj_ativos + mercado.qt_socios_pj_nulos + mercado.qt_socios_pj_baixados + mercado.qt_socios_pj_suspensos + mercado.qt_socios_pj_inaptos
mercado['prop_sociospj_inativos']  = (mercado.qt_socios_pj_nulos + mercado.qt_socios_pj_baixados + mercado.qt_socios_pj_suspensos + mercado.qt_socios_pj_inaptos) / mercado.qt_total_socios_pj
mercado['prop_sociospj_ativos']    = mercado.qt_socios_pj_ativos / mercado.qt_total_socios_pj
mercado['prop_coligados_inativos'] = (mercado.qt_coligados_baixada + mercado.qt_coligados_inapta + mercado.qt_coligados_suspensa + mercado.qt_coligados_nula) / mercado.qt_coligados
mercado['prop_coligados_ativos']   = mercado.qt_coligados_ativo / mercado.qt_coligados
mercado['media_fat_est_colig_gp_unid_folha_pgto'] = mercado.media_faturamento_est_coligados_gp / mercado.media_vl_folha_coligados_gp
mercado['media_fat_est_colig_unid_folha_pgto']    = mercado.media_faturamento_est_coligados / mercado.media_vl_folha_coligados
mercado['qt_coligados_atividade_ruim'] = mercado.qt_coligados_atividade_baixo + mercado.qt_coligados_atividade_mt_baixo + mercado.qt_coligados_atividade_inativo
mercado['fatur_por_coligada']    = mercado.sum_faturamento_estimado_coligadas / mercado.qt_coligadas
mercado['proporcao_desligados']  = mercado.qt_desligados / mercado.qt_funcionarios
mercado['proporcao_contratados'] = mercado.qt_admitidos / mercado.qt_funcionarios

# conjunto de dados portfolio1
portfolio1['qt_total_socios_pj']      = portfolio1.qt_socios_pj_ativos + portfolio1.qt_socios_pj_nulos + portfolio1.qt_socios_pj_baixados + portfolio1.qt_socios_pj_suspensos + portfolio1.qt_socios_pj_inaptos
portfolio1['prop_sociospj_inativos']  = (portfolio1.qt_socios_pj_nulos + portfolio1.qt_socios_pj_baixados + portfolio1.qt_socios_pj_suspensos + portfolio1.qt_socios_pj_inaptos) / portfolio1.qt_total_socios_pj
portfolio1['prop_sociospj_ativos']    = portfolio1.qt_socios_pj_ativos / portfolio1.qt_total_socios_pj
portfolio1['prop_coligados_inativos'] = (portfolio1.qt_coligados_baixada + portfolio1.qt_coligados_inapta + portfolio1.qt_coligados_suspensa + portfolio1.qt_coligados_nula) / portfolio1.qt_coligados
portfolio1['prop_coligados_ativos']   = portfolio1.qt_coligados_ativo / portfolio1.qt_coligados
portfolio1['media_fat_est_colig_gp_unid_folha_pgto'] = portfolio1.media_faturamento_est_coligados_gp / portfolio1.media_vl_folha_coligados_gp
portfolio1['media_fat_est_colig_unid_folha_pgto']    = portfolio1.media_faturamento_est_coligados / portfolio1.media_vl_folha_coligados
portfolio1['qt_coligados_atividade_ruim'] = portfolio1.qt_coligados_atividade_baixo + portfolio1.qt_coligados_atividade_mt_baixo + portfolio1.qt_coligados_atividade_inativo
portfolio1['fatur_por_coligada']    = portfolio1.sum_faturamento_estimado_coligadas / portfolio1.qt_coligadas
portfolio1['proporcao_desligados']  = portfolio1.qt_desligados / portfolio1.qt_funcionarios
portfolio1['proporcao_contratados'] = portfolio1.qt_admitidos / portfolio1.qt_funcionarios

# conjunto de dados portfolio2
portfolio2['qt_total_socios_pj']      = portfolio2.qt_socios_pj_ativos + portfolio2.qt_socios_pj_nulos + portfolio2.qt_socios_pj_baixados + portfolio2.qt_socios_pj_suspensos + portfolio2.qt_socios_pj_inaptos
portfolio2['prop_sociospj_inativos']  = (portfolio2.qt_socios_pj_nulos + portfolio2.qt_socios_pj_baixados + portfolio2.qt_socios_pj_suspensos + portfolio2.qt_socios_pj_inaptos) / portfolio2.qt_total_socios_pj
portfolio2['prop_sociospj_ativos']    = portfolio2.qt_socios_pj_ativos / portfolio2.qt_total_socios_pj
portfolio2['prop_coligados_inativos'] = (portfolio2.qt_coligados_baixada + portfolio2.qt_coligados_inapta + portfolio2.qt_coligados_suspensa + portfolio2.qt_coligados_nula) / portfolio2.qt_coligados
portfolio2['prop_coligados_ativos']   = portfolio2.qt_coligados_ativo / portfolio2.qt_coligados
portfolio2['media_fat_est_colig_gp_unid_folha_pgto'] = portfolio2.media_faturamento_est_coligados_gp / portfolio2.media_vl_folha_coligados_gp
portfolio2['media_fat_est_colig_unid_folha_pgto']    = portfolio2.media_faturamento_est_coligados / portfolio2.media_vl_folha_coligados
portfolio2['qt_coligados_atividade_ruim'] = portfolio2.qt_coligados_atividade_baixo + portfolio2.qt_coligados_atividade_mt_baixo + portfolio2.qt_coligados_atividade_inativo
portfolio2['fatur_por_coligada']    = portfolio2.sum_faturamento_estimado_coligadas / portfolio2.qt_coligadas
portfolio2['proporcao_desligados']  = portfolio2.qt_desligados / portfolio2.qt_funcionarios
portfolio2['proporcao_contratados'] = portfolio2.qt_admitidos / portfolio2.qt_funcionarios

# conjunto de dados portfolio3
portfolio3['qt_total_socios_pj']      = portfolio3.qt_socios_pj_ativos + portfolio3.qt_socios_pj_nulos + portfolio3.qt_socios_pj_baixados + portfolio3.qt_socios_pj_suspensos + portfolio3.qt_socios_pj_inaptos
portfolio3['prop_sociospj_inativos']  = (portfolio3.qt_socios_pj_nulos + portfolio3.qt_socios_pj_baixados + portfolio3.qt_socios_pj_suspensos + portfolio3.qt_socios_pj_inaptos) / portfolio3.qt_total_socios_pj
portfolio3['prop_sociospj_ativos']    = portfolio3.qt_socios_pj_ativos / portfolio3.qt_total_socios_pj
portfolio3['prop_coligados_inativos'] = (portfolio3.qt_coligados_baixada + portfolio3.qt_coligados_inapta + portfolio3.qt_coligados_suspensa + portfolio3.qt_coligados_nula) / portfolio3.qt_coligados
portfolio3['prop_coligados_ativos']   = portfolio3.qt_coligados_ativo / portfolio3.qt_coligados
portfolio3['media_fat_est_colig_gp_unid_folha_pgto'] = portfolio3.media_faturamento_est_coligados_gp / portfolio3.media_vl_folha_coligados_gp
portfolio3['media_fat_est_colig_unid_folha_pgto']    = portfolio3.media_faturamento_est_coligados / portfolio3.media_vl_folha_coligados
portfolio3['qt_coligados_atividade_ruim'] = portfolio3.qt_coligados_atividade_baixo + portfolio3.qt_coligados_atividade_mt_baixo + portfolio3.qt_coligados_atividade_inativo
portfolio3['fatur_por_coligada']    = portfolio3.sum_faturamento_estimado_coligadas / portfolio3.qt_coligadas
portfolio3['proporcao_desligados']  = portfolio3.qt_desligados / portfolio3.qt_funcionarios
portfolio3['proporcao_contratados'] = portfolio3.qt_admitidos / portfolio3.qt_funcionarios

In [ ]:
ls_cols_excluir = ['qt_socios_pj_ativos', 'qt_socios_pj_nulos', 'qt_socios_pj_baixados', 'qt_socios_pj_suspensos', 'qt_socios_pj_inaptos', 
                   'qt_coligados_baixada', 'qt_coligados_inapta', 'qt_coligados_suspensa', 'qt_coligados_nula', 'qt_coligados_atividade_baixo', 
                   'qt_coligados_atividade_mt_baixo', 'qt_desligados', 'qt_admitidos', 'qt_socios_pf', 'qt_socios_pj', 'de_faixa_faturamento_estimado', 
                   'vl_faturamento_estimado_aux', 'fl_optante_simples', 'fl_optante_simei', 'qt_coligados_ativo']

mercado.drop(ls_cols_excluir, axis=1, inplace=True)
portfolio1.drop(ls_cols_excluir, axis=1, inplace=True)
portfolio2.drop(ls_cols_excluir, axis=1, inplace=True)
portfolio3.drop(ls_cols_excluir, axis=1, inplace=True)

In [ ]:
# excluir colunas cuja proporção de valores ausentes ultrapasse 10%
df_missing = missing_values_table(mercado)
ls_cols_10 = df_missing[df_missing['% of Total Values'] > 10].index

mercado.drop(ls_cols_10, axis=1, inplace=True)
portfolio1.drop(ls_cols_10, axis=1, inplace=True)
portfolio2.drop(ls_cols_10, axis=1, inplace=True)
portfolio3.drop(ls_cols_10, axis=1, inplace=True)

In [ ]:
# gerar lista de colunas numéricas com valores ausentes para substituir estes valores pela sua mediana
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
ls_var_numerica = list(mercado.select_dtypes(include=numerics).columns)

from sklearn.impute import SimpleImputer
imp_median = SimpleImputer(strategy='median')
imp_median.fit(mercado[ls_var_numerica])

mercado[ls_var_numerica]    = imp_median.transform(mercado[ls_var_numerica])
portfolio1[ls_var_numerica] = imp_median.transform(portfolio1[ls_var_numerica])
portfolio2[ls_var_numerica] = imp_median.transform(portfolio2[ls_var_numerica])
portfolio3[ls_var_numerica] = imp_median.transform(portfolio3[ls_var_numerica])

In [ ]:
# identificar os tipos de dados das colunas ainda com valores ausentes
df_missing = missing_values_table(mercado)
ls_cols_missing = df_missing.index

# aplicar aos conjuntos de dados
for col in ls_cols_missing:
    mais_frequente = mercado[col].mode()
    mercado[col].fillna(mais_frequente[0], inplace=True)

    if portfolio1[col].isna().sum() > 0:
        portfolio1[col].fillna(mais_frequente[0], inplace=True)

    if portfolio2[col].isna().sum() > 0:
        portfolio2[col].fillna(mais_frequente[0], inplace=True)

    if portfolio3[col].isna().sum() > 0:
        portfolio3[col].fillna(mais_frequente[0], inplace=True)

In [ ]:
missing_values_table(mercado)

In [ ]:
missing_values_table(portfolio1)

In [ ]:
missing_values_table(portfolio2)

In [ ]:
missing_values_table(portfolio3)

In [ ]:
# Substituir o valor 'MUITO BAIXA' por 'BAIXA'

mercado['de_nivel_atividade']    = mercado['de_nivel_atividade'].replace(to_replace = {'MUITO BAIXA': 'BAIXA'})
portfolio1['de_nivel_atividade'] = portfolio1['de_nivel_atividade'].replace(to_replace = {'MUITO BAIXA': 'BAIXA'})
portfolio2['de_nivel_atividade'] = portfolio2['de_nivel_atividade'].replace(to_replace = {'MUITO BAIXA': 'BAIXA'})
portfolio3['de_nivel_atividade'] = portfolio3['de_nivel_atividade'].replace(to_replace = {'MUITO BAIXA': 'BAIXA'})

Anexar os clusters aos conjuntos de dados

In [ ]:
mercado['cluster']    = mercado_clusters
portfolio1['cluster'] = portfolio1_clusters
portfolio2['cluster'] = portfolio2_clusters
portfolio3['cluster'] = portfolio3_clusters

Anexar as distâncias para os clusters aos conjuntos de dados

In [ ]:
mercado['dist_centro']    = mercado_dist
portfolio1['dist_centro'] = portfolio1_dist
portfolio2['dist_centro'] = portfolio2_dist
portfolio3['dist_centro'] = portfolio3_dist

Fazer algumas visualizações no conjunto de dados Mercado para verificar o resultado do Kmeans

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(30,5))
sns.countplot(data = mercado, y = 'fl_st_especial', hue = 'cluster').set_title('fl_st_especial')

In [ ]:
plt.figure(figsize=(30,5))
sns.countplot(data = mercado, y = 'fl_email', hue = 'cluster').set_title('fl_email')

In [ ]:
plt.figure(figsize=(30,5))
sns.countplot(data = mercado, y = 'fl_telefone', hue = 'cluster').set_title('fl_telefone')

In [ ]:
plt.figure(figsize=(30,5))
sns.countplot(data = mercado, y = 'fl_spa', hue = 'cluster').set_title('fl_spa')

In [ ]:
plt.figure(figsize=(30,5))
sns.countplot(data = mercado, y = 'fl_antt', hue = 'cluster').set_title('fl_antt')

In [ ]:
plt.figure(figsize=(30,5))
sns.countplot(data = mercado, y = 'fl_simples_irregular', hue = 'cluster').set_title('fl_simples_irregular')

In [ ]:
plt.figure(figsize=(30,5))
sns.countplot(data = mercado, y = 'fl_rm', hue = 'cluster').set_title('fl_rm')

In [ ]:
plt.figure(figsize=(28,10))
ax = sns.countplot(data=mercado, x='sg_uf_matriz', hue = 'cluster')

In [ ]:
plt.figure(figsize=(28,10))
ax = sns.countplot(data=mercado, x='sg_uf', hue = 'cluster')

In [ ]:
mercado['de_saude_rescencia']    = mercado['de_saude_rescencia'].replace(to_replace = {'ATE 3 MESES': 'ATE 6 MESES'})
portfolio1['de_saude_rescencia'] = portfolio1['de_saude_rescencia'].replace(to_replace = {'ATE 3 MESES': 'ATE 6 MESES'})
portfolio2['de_saude_rescencia'] = portfolio2['de_saude_rescencia'].replace(to_replace = {'ATE 3 MESES': 'ATE 6 MESES'})
portfolio3['de_saude_rescencia'] = portfolio3['de_saude_rescencia'].replace(to_replace = {'ATE 3 MESES': 'ATE 6 MESES'})

In [ ]:
# visualizar a diferença
plt.figure(figsize=(28,10))
ax = sns.countplot(data=mercado, x='de_saude_rescencia', hue='cluster')

In [ ]:
plt.figure(figsize=(28,10))
ax = sns.countplot(data=mercado, x='de_saude_tributaria', hue='cluster')

In [ ]:
# visualizar o resultado

plt.figure(figsize=(30, 5))
ax = sns.countplot(data=mercado, y = 'de_nivel_atividade', hue = 'cluster')

### DBScan

O modelo DBScan é uma alternativa ao K-means. O problema do DBScan é que ele pode classificar pontos como ruído, ao invés de atribuí-lo a um cluster. 

In [ ]:
from sklearn.cluster import DBSCAN

dbscan_v1 = DBSCAN()

# Fit
dbscan_v1.fit(X)

# Labels
unique, counts = np.unique(dbscan_v1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

O modelo DBSCAN encontrou grande quantidade de clusters.